https://www.gutenberg.org/files/11/11-0.txt

# WebScraping

In [1]:
!pip install utils

In [7]:
import pandas as pd
import urllib
import requests
from bs4 import BeautifulSoup
import re

In [8]:
url = 'https://www.gutenberg.org/files/11/11-0.txt'
res = requests.get(url)

In [9]:
html_page = res.content

In [10]:
type(html_page)

bytes

In [11]:
soup = BeautifulSoup(html_page, features ='html.parser')

In [ ]:
soup # executing this line will show all the text scraped from the webstie

'''
The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

'''

In [22]:
type(soup)

bs4.BeautifulSoup

In [23]:
text = soup.find_all(text = True)
type(text)

bs4.element.ResultSet

In [24]:
# to get line by line or to get the split the sentence with .
lines = [line2 for line in text for line2 in line.strip().split('.') if line2]

In [25]:
df = pd.DataFrame(lines, columns = ['Text'])

In [26]:
df.head(2)

,Text
0,The Project Gutenberg eBook of Alice’s Adventu...
1,"You may copy it, give it away or re-use it un..."


In [27]:
df.replace(to_replace=[r'\\t|\\n|\\r', '\t|\n|\r'], value = ['',''], regex = True, inplace = True)


In [28]:
df = df.rename_axis("Id", axis='columns')

In [29]:
df.to_csv('Alice.csv', index_label = "Id")

# Sentence similarity

In [30]:
import pandas as pd
import os
import gensim
import spacy

In [31]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [32]:
data = pd.read_csv('Alice.csv')

In [33]:
data.head(3)

,Id,Text
0,0,The Project Gutenberg eBook of Alice’s Adventu...
1,1,"You may copy it, give it away or re-use it un..."
2,2,gutenberg


we will select a random particular row, for which, that would be our source sentence for which will be finding five or 10 similar sentences. 

In [34]:
ref_sentence = data.loc[data['Id']==748,'Text'].iloc[0]

In [35]:
ref_sentence

'“It must be a very pretty dance,” said Alice timidly'

In [36]:
ref_sentence_vec = nlp(ref_sentence) # vectorizing tokens

In [37]:
all_docs = [nlp(row) for row in data['Text']]

In [38]:
sims = []
doc_id = []
for i in range (len(all_docs)):
    sim = all_docs[i].similarity(ref_sentence_vec)
    sims.append(sim)
    doc_id.append(i)
    sims_docs = pd.DataFrame(list(zip(doc_id, sims)), columns = ['doc_id','sims'])

C:\Users\shykat\AppData\Local\Temp\ipykernel_19504\3304570205.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  sim = all_docs[i].similarity(ref_sentence_vec)
C:\Users\shykat\AppData\Local\Temp\ipykernel_19504\3304570205.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive te

In [39]:
sims_docs_sorted = sims_docs.sort_values(by='sims', ascending = False)

In [40]:
top_5_sim_docs = data.iloc[sims_docs_sorted['doc_id'][1:6]]

In [41]:
print(data[data["Id"]==748]['Text'].values)

['“It must be a very pretty dance,” said Alice timidly']


In [42]:
top_sim_scores = pd.concat([top_5_sim_docs,sims_docs_sorted['sims'][1:6]],axis = 1)

In [43]:
for (text, sim) in zip(top_sim_scores['Text'], top_sim_scores['sims']):
    print("The top 5 similar sentences are:{}\n with a similarity scores of {:.2f}\n".format(text, sim))

The top 5 similar sentences are:“She can’t explain it,” said the Gryphon hastily
 with a similarity scores of 0.79

The top 5 similar sentences are:“I’ve a right to think,” said Alice sharply, for she was beginning tofeel a little worried
 with a similarity scores of 0.76

The top 5 similar sentences are:“Then it wasn’t very civil of you to offer it,” said Alice angrily
 with a similarity scores of 0.75

The top 5 similar sentences are:“The trial cannot proceed,” said the King in a very grave voice, “untilall the jurymen are back in their proper places—_all_,” he repeatedwith great emphasis, looking hard at Alice as he said so
 with a similarity scores of 0.73

The top 5 similar sentences are:Will you, won’t you, will you, won’t you, will you join the dance?Will you, won’t you, will you, won’t you, won’t you join the dance?”“Thank you, it’s a very interesting dance to watch,” said Alice,feeling very glad that it was over at last: “and I do so like thatcurious song about the whiting!”“O